### Get a player's name and scrape their infos from Leaguepedia ###

In [1]:
from bs4 import BeautifulSoup as bs
import requests as rq
import re
import pandas as pd

In [2]:
url = "https://lol.fandom.com/wiki/"
player_ID = "Faker"

full_url = url + player_ID

r = rq.get(full_url)

player_page = bs(r.content)
print(player_page.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Faker - Leaguepedia | League of Legends Esports Wiki
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Faker","wgTitle":"Faker","wgCurRevisionId":3186255,"wgRevisionId":3186255,"wgArticleId":158918,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["PlayersWithAutomatedNews","PlayersWithAutomatedGallery","Players","Korean Residents","Players Born In 1996","Players Born in South Korea"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgD

In [3]:
# Scrape the info table
infos_table = player_page.find_all("table", attrs={"id":"infoboxPlayer"})[0]
print(infos_table.prettify())

<table class="infobox infobox-player-narrow InfoboxPlayer" id="infoboxPlayer">
 <tbody>
  <tr>
   <th class="infobox-title" colspan="2">
    Faker
   </th>
  </tr>
  <tr class="mw-empty-elt">
  </tr>
  <tr>
   <td class="infobox-wide" colspan="2">
    <div class="center">
     <div class="floatnone">
      <a class="image" href="https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/0/00/T1_Faker_2022_Split_1.png/revision/latest?cb=20220128072310">
       <img alt="T1 Faker 2022 Split 1.png" class="lazyload" data-file-height="278" data-file-width="375" data-image-key="T1_Faker_2022_Split_1.png" data-image-name="T1 Faker 2022 Split 1.png" data-src="https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/0/00/T1_Faker_2022_Split_1.png/revision/latest/scale-to-width-down/220?cb=20220128072310" decoding="async" height="163" src="data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D" width="220"/>
      </a>
     </div>
    </div>
   </td>

In [26]:
# Scrape all info labels
infos_boxes = infos_table.select("td.infobox-label")
infos_labels = ["Player ID"] + [label.string for label in infos_boxes if label]
infos_labels

['Player ID',
 'Name',
 'Country of Birth',
 'Birthday',
 'Residency',
 'Team',
 'Contract Expires',
 'Role',
 'Favorite Champs',
 'Soloqueue IDs']

In [47]:
# Scrape all infos
infos = [player_ID] + [label.next_sibling.get_text() for label in infos_boxes]

# Scrape favorite champs
champ_list = infos_table.select("span.champion-sprite")
fav_champs = [champ["title"] for champ in champ_list]
fav_champs

# Add favorite champs to infos
fav_champs_str = ""
for chmp in fav_champs:
    fav_champs_str += (chmp + ",")
    
infos[8] = fav_champs_str 
infos

['Faker',
 'Lee Sang-hyeok (이상혁)',
 'South Korea',
 'May 7, 1996 (age 26)',
 'KRKorea',
 '\u2060\u2060T1',
 '2022-11-21',
 'Mid Laner',
 'Zed,LeBlanc,Ahri,Azir,Ryze,Twisted Fate,Yasuo,',
 'KR: Hide on bushEUW: Koearnhide on streamwincgNA: mid fekarCN: dakjshdkjBR: 19960507VN: mid24']

In [48]:
# Make a panda table

# Make columns
cols_names = infos_labels
cols_names
print(len(cols_names))

# Make rows
rows = infos
print(len(rows))

# Make the table
df = pd.DataFrame([rows], columns=cols_names)
df

10
10


Player ID                  Name Country of Birth              Birthday  \
0     Faker  Lee Sang-hyeok (이상혁)      South Korea  May 7, 1996 (age 26)   

  Residency  Team Contract Expires       Role  \
0   KRKorea  ⁠⁠T1       2022-11-21  Mid Laner   

                                  Favorite Champs  \
0  Zed,LeBlanc,Ahri,Azir,Ryze,Twisted Fate,Yasuo,   

                                       Soloqueue IDs  
0  KR: Hide on bushEUW: Koearnhide on streamwincg...